In [ ]:

import itertools
import numpy as np
import os
import seaborn as sns
from tqdm import tqdm
from dataclasses import asdict, dataclass, field
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
import matplotlib.pyplot as plt
import pandas as pd

import vpype_cli
from typing import List, Generic
from genpen import genpen as gp
from genpen.utils import Paper
from scipy import stats as ss
import geopandas
from shapely.errors import TopologicalError
import functools
import vpype
from skimage import io
from pathlib import Path

from sklearn.preprocessing import minmax_scale
from skimage import feature
from skimage import exposure

from skimage import filters
from skimage.color import rgb2gray
from skimage.transform import rescale, resize, downscale_local_mean
from skimage.morphology import disk
from pyaxidraw import axidraw   # import module

from PIL import Image

import fn
import cv2

from genpen.flow.field import *
from genpen.flow.particle import *
from genpen.genpen import random_line_subdivide_gen

import time
from pov import pov

%load_ext autoreload
%autoreload 2

In [ ]:
def shoot_it(linestrings, gun_pts, substring_start_ds, substring_len_ds):
    substrings = []
    for ii, ls in enumerate(linestrings):
        endpt = ls.interpolate(np.random.uniform(), normalized=True)
        connection = LineString((gun_pts[ii], endpt))
        substring_start_d = substring_start_ds()
        substring_end_d = substring_start_d + substring_len_ds()
        substring_start = connection.interpolate(substring_start_d, normalized=True)
        substring_end = connection.interpolate(substring_end_d, normalized=True)
        substring = LineString((substring_start, substring_end))
        substrings.append(substring)
    substrings = gp.merge_LineStrings(substrings)
    return substrings

# more complex

In [ ]:

paper_size:str = '6x5.99 inches'
border:float=15  # mm
savedir='/home/naka/art/plotter_svgs'

In [ ]:
# make page
paper = Paper(paper_size)
drawbox = paper.get_drawbox(border)


In [ ]:
center = drawbox.centroid

In [ ]:
db = gp.Shape(drawbox)

In [ ]:
corner = Point(35,center.y)

In [ ]:
n_circles = 1
xcenter = 100
yrange = np.linspace(db.bottom+30, db.top-30, n_circles)

In [ ]:
pts = [Point(xcenter, y) for y in yrange]

In [ ]:
radmin = 50
radmax = 50
rads = np.linspace(radmin, radmax, len(pts))
circles = [pt.buffer(d) for d,pt in zip(rads, pts)]

In [ ]:
prms = gp.ScaleTransPrms(n_iters=300)

In [ ]:
cs = [gp.Poly(c) for c in circles]

In [ ]:
for c in cs:
    c.fill_hatch(angle=0, spacing=0.35)

In [ ]:
fillsets = [c.fill for c in cs]
shotsets = []
for fills in fillsets:
    gun_pts = [corner] * len(fills)
    substring_start_ds = ss.uniform(loc=0.05, scale=0.25).rvs
    substring_len_ds = ss.uniform(loc=0.4, scale=0.3).rvs
    shots = shoot_it(
        linestrings=fills, 
        gun_pts=gun_pts, 
        substring_start_ds=substring_start_ds, 
        substring_len_ds=substring_len_ds)
    shotsets.append(shots)

In [ ]:


   
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
counter = 1

for shots, fills in zip(shotsets, fillsets):
    for s,f, in zip(shots, fills):
        sk.stroke(counter)
        sk.geometry(s)
        counter += 1
        sk.stroke(counter)
        sk.geometry(f)
        counter += 1
    
    
# tolerance=0.5
sk.display()

In [ ]:
plot_id = fn.new_plot_id()

In [ ]:
savepath = Path(savedir).joinpath(f'{plot_id}.svg').as_posix()
sk.save(savepath)

In [ ]:
import signal

class GracefulExiter():

    def __init__(self):
        self.state = False
        signal.signal(signal.SIGINT, self.change_state)

    def change_state(self, signum, frame):
        print("exit flag set to True (repeat to exit now)")
        signal.signal(signal.SIGINT, signal.SIG_DFL)
        self.state = True

    def exit(self):
        return self.state

In [ ]:
def plot_layer(ad, cam, layer_number, wait_time=1.,):
    ad.options.layer = ii
    ad.plot_run()
    time.sleep(wait_time)
    cam.save_image()

In [ ]:
doc = vpype.read_multilayer_svg(savepath, 0.1)

In [ ]:
n_layers = len(doc.layers)
print(n_layers)

In [ ]:
wait_time = 2.1

In [ ]:
from contextlib import closing
with closing(pov.Camera()) as cam:
    img = cam.get_image()

In [ ]:
cam = pov.Camera()

In [ ]:
cam.preview()

In [ ]:
ad = axidraw.AxiDraw()   
ad.plot_setup(savepath)
ad.options.mode = "layers"
ad.options.units = 2
ad.options.speed_pendown = 60
ad.options.speed_penup = 60
ad.update()

In [ ]:
timing_info = []
stored_exception = None
flag = GracefulExiter()
for ii in tqdm(range(n_layers)):
    timing_info.append(plot_layer(ad, cam, layer_number=ii, wait_time=2.1))
    if flag.exit():
        break

In [ ]:
cam.close()

In [ ]:
plot_id = fn.get_current_plot_id()

In [ ]:
Path(gp.SVG_SAVEDIR).joinpath(plot_id).with_suffix('.svg')

# trying refactored version

In [ ]:
def substring(linestring, substring_start, substring_end):
    return LineString((linestring.interpolate(substring_start, normalized=True), linestring.interpolate(substring_end, normalized=True)))

In [ ]:
def substrings(linestrings, substring_starts, substring_ends):
    _substring_starts = gp.make_callable(substring_starts)
    _substring_ends = gp.make_callable(substring_ends)
    substrings = []
    for ii, ls in enumerate(linestrings):
        start = _substring_starts()
        end = _substring_ends()
        substrings.append(substring(ls, start, end))
    substrings = gp.merge_LineStrings(substrings)
    return substrings

In [ ]:
db = gp.Poly(drawbox)

In [ ]:
db.top

In [ ]:
db.p.centroid.y

In [ ]:
np.linspace(46, 106, n_lines)

In [ ]:
@dataclass
class Streamers:
    n_lines: int = 30
    xstarts: 'typing.any' = 0.
    ystarts: 'typing.any' = 0.
    xends: 'typing.any' = 1.
    yends: 'typing.any' = 1.
        
    @property
    def lss(self):
        xstarts = gp.make_callable(self.xstarts)
        ystarts = gp.make_callable(self.ystarts)
        xends = gp.make_callable(self.xends)
        yends = gp.make_callable(self.yends)
        lss = []
        for ii in range(n_lines):
            pt0 = Point(xstarts(), ystarts())
            pt1 = Point(xends(), yends())
            ls = LineString((pt0, pt1))
            lss.append(ls)

        lss = gp.merge_LineStrings(lss)
        return lss
    
@dataclass
class Blasters:
    lss: sg.MultiLineString = None
    n_substring_ints:int = 30
    substring_var_start: 'typing.any' = 0.
    substring_var_end: 'typing.any' = 0.1
    substring_len: 'typing.any' = 0.05
    xjitter: 'typing.any' = 0.
    yjitter: 'typing.any' = 0.
        
    @property
    def blss(self):
        xjitter = gp.make_callable(self.xjitter)
        yjitter = gp.make_callable(self.yjitter)
        blss = []
        for start in np.linspace(0., 1., self.n_substring_ints):
            substring_starts = ss.uniform(loc=start, scale=np.interp(start, [0, 1], [self.substring_var_start, self.substring_var_end])).rvs
            substring_ends = ss.uniform(loc=start+self.substring_len, scale=np.interp(start, [0, 1], [self.substring_var_start, self.substring_var_end])).rvs
            bls = substrings(self.lss, substring_starts, substring_ends)
            bls = [sa.translate(ls, xoff=xjitter(), yoff=yjitter()) for ls in bls]
            blss.append(bls)
        return blss
    
    

In [ ]:
lss0 = Streamers(n_lines=30, xstarts=15, xends=76, ystarts=list(np.linspace(26, 126, n_lines)), yends=list(np.linspace(66, 86, n_lines))).lss
blss0 = Blasters(lss=lss0, n_substring_ints=20, substring_var_start=0.0, 
                 substring_var_end=0.04, substring_len=0.05, yjitter = ss.norm(loc=0, scale=1).rvs,
                ).blss

In [ ]:
lss1 = Streamers(n_lines=30, xstarts=137, xends=76, ystarts=list(np.linspace(26, 126, n_lines)), yends=list(np.linspace(66, 86, n_lines))).lss
blss1 = Blasters(lss=lss1, n_substring_ints=20, substring_var_start=0.0, 
                 substring_var_end=0.04, substring_len=0.05, yjitter = ss.norm(loc=0, scale=1).rvs,
                ).blss

In [ ]:
n_splits_per_blast = 2
buffer_dist = 1.5
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
counter = 1

for b0, b1 in zip(blss0, blss1):
    
    splits = np.split(np.random.permutation(np.arange(len(b0))), n_splits_per_blast)
    for order in splits:
        for ii in order:
            s = b0[ii]
            sk.stroke(counter)
            sk.geometry(s)
        counter += 1
        for ii in order:
            s = b1[ii]
            sk.stroke(counter)
            sk.geometry(s)
        counter += 1
    
    for order in splits:
        
        poly = gp.merge_LineStrings([b0[ii] for ii in order]).buffer(buffer_dist, cap_style=1, join_style=1)
        P = gp.Poly(poly)
        prms = gp.ScaleTransPrms(
            n_iters=200,
            d_buffer=-0.35,
            d_translate_factor=0.7,
            angles=0,
        )
        P.fill_scale_trans(**prms.prms)
        fill_splits = np.array_split(np.arange(len(P.fill)), n_fill_splits)
        for fill_split in fill_splits:
            for ii in fill_split:
                s = P.fill[ii]
                sk.stroke(counter)
                sk.geometry(s)
            counter += 1
            
        poly = gp.merge_LineStrings([b1[ii] for ii in order]).buffer(buffer_dist, cap_style=2, join_style=2)
        P = gp.Poly(poly)
        prms = gp.ScaleTransPrms(
            n_iters=200,
            d_buffer=-0.35,
            d_translate_factor=0.7,
            angles=0,
        )
        P.fill_scale_trans(**prms.prms)
        fill_splits = np.array_split(np.arange(len(P.fill)), n_fill_splits)
        for fill_split in fill_splits:
            for ii in fill_split:
                s = P.fill[ii]
                sk.stroke(counter)
                sk.geometry(s)
            counter += 1
    
sk.display()

In [ ]:
counter

In [ ]:
plot_id = fn.new_plot_id()

In [ ]:
savepath = Path(savedir).joinpath(f'{plot_id}.svg').as_posix()
sk.save(savepath)

## plot

In [ ]:
from genpen.axicam import AxiCam

In [ ]:
axidraw.AxiDraw()

In [ ]:
cam

In [ ]:
ac = AxiCam()

In [ ]:
ac.init_cam()

In [ ]:
ac.plot_layers(wait_times=0.5)

# try2

In [ ]:
lss0 = Streamers(n_lines=60, xstarts=15, xends=137, ystarts=list(np.linspace(16, 136, n_lines)), yends=list(np.linspace(71, 81, n_lines))).lss
blss0 = Blasters(lss=lss0, n_substring_ints=50, substring_var_start=0.0, substring_var_end=0.0, substring_len=0.02,
                 yjitter = ss.norm(loc=0, scale=0.).rvs, xjitter = ss.norm(loc=0, scale=0).rvs, 
                 ).blss

In [ ]:
lss1 = Streamers(n_lines=70, xstarts=137, xends=15, ystarts=list(np.linspace(16, 136, n_lines)), yends=list(np.linspace(66, 86, n_lines))).lss
blss1 = Blasters(lss=lss1, n_substring_ints=50, substring_var_start=0.0, substring_var_end=0.0, substring_len=0.02,
                 yjitter = ss.norm(loc=0, scale=0.).rvs, xjitter = ss.norm(loc=0, scale=0).rvs, 
                 ).blss

In [ ]:
n_splits_per_blast = 9
n_fill_splits = 1
buffer_dist = 4.5
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
counter = 1

splits = [np.array_split(np.random.permutation(np.arange(len(b0))), n_splits_per_blast) for b0 in blss0]

for split, b0, b1 in zip(splits, blss0, blss1):
    
    for order in split:
        for ii in order:
            s = b0[ii]
            sk.stroke(counter)
            sk.geometry(s)
        counter += 1
        for ii in order:
            s = b1[ii]
            sk.stroke(counter)
            sk.geometry(s)
        counter += 1
    
sk.display()
sk.vpype('linesort')

In [ ]:
counter

In [ ]:
plot_id = fn.new_plot_id()

In [ ]:
savepath = Path(savedir).joinpath(f'{plot_id}.svg').as_posix()
sk.save(savepath)

## plot

In [ ]:
from genpen.axicam import AxiCam

In [ ]:
axidraw.AxiDraw()

In [ ]:
cam

In [ ]:
ac = AxiCam()

In [ ]:
ac.init_cam()

In [ ]:
ac.ad.options.speed_pendown = 40
ac.ad.options.pen_delay_up = 5
ac.ad.update()

In [ ]:
ac.plot_layers(wait_times=0.5)

# try3

In [ ]:
lss0 = Streamers(n_lines=40, xstarts=15, xends=137, ystarts=list(np.linspace(16, 136, n_lines)), yends=list(np.linspace(56, 96, n_lines))).lss
blss0 = Blasters(lss=lss0, n_substring_ints=70, substring_var_start=0.0, yjitter = ss.norm(loc=0, scale=1).rvs, xjitter = ss.norm(loc=0, scale=1).rvs, 
                 substring_var_end=0.05, substring_len=0.05).blss

In [ ]:
lss1 = Streamers(n_lines=20, xstarts=137, xends=15, ystarts=list(np.linspace(16, 136, n_lines)), yends=list(np.linspace(66, 86, n_lines))).lss
blss1 = Blasters(lss=lss1, n_substring_ints=70, substring_var_start=0.0, yjitter = ss.norm(loc=0, scale=0.5).rvs, xjitter = ss.norm(loc=0, scale=0).rvs, 
                 substring_var_end=0.05, substring_len=0.05).blss

In [ ]:
n_splits_per_blast = 6
n_fill_splits = 1
buffer_dist = 4.5
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
counter = 1

splits = [np.array_split(np.random.permutation(np.arange(len(b0))), n_splits_per_blast) for b0 in blss0]

for split, b0, b1 in zip(splits, blss0, blss1):
    
    for order in split:
        for ii in order:
            s = b0[ii]
            sk.stroke(counter)
            sk.geometry(s)
        counter += 1
        for ii in order:
            s = b1[ii]
            sk.stroke(counter)
            sk.geometry(s)
        counter += 1
    
sk.display()

In [ ]:
counter

In [ ]:
plot_id = fn.new_plot_id()

In [ ]:
savepath = Path(savedir).joinpath(f'{plot_id}.svg').as_posix()
sk.save(savepath)

## plot

In [ ]:
from genpen.axicam import AxiCam

In [ ]:
axidraw.AxiDraw()

In [ ]:
cam

In [ ]:
ac = AxiCam()

In [ ]:
ac.init_cam()

In [ ]:
ac.plot_layers(wait_times=0.5)